# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 65 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2311.11534


extracting tarball to tmp_2311.11534...

 done.
Retrieving document from  https://arxiv.org/e-print/2311.11588


extracting tarball to tmp_2311.11588...

...

✔ → 200451:\section{SMA data}
  ↳ 229432:\begin{appendix}
✔ → 229432:\begin{appendix}
  ↳ 229449:\section{IRAM 30 m data}
✔ → 229449:\section{IRAM 30 m data}
  ↳ 232377:\section{SMA data}


✔ → 232377:\section{SMA data}
  ↳ 234915:end


list index out of range
Retrieving document from  https://arxiv.org/e-print/2311.11909


extracting tarball to tmp_2311.11909...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.11832-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.11832) | **A$^{3}$COSMOS: Dissecting the gas content of star-forming galaxies  across the main sequence at 1.2 $\leq z$ < 1.6**  |
|| T.-M. Wang, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-11-21*|
|*Comments*| *20 pages, 17 figures*|
|**Abstract**| We aim to understand the physical mechanisms that drive star formation in a sample of mass-complete (>10$^{9.5}M_{\odot}$) star-forming galaxies (SFGs) at 1.2 $\leq z$ < 1.6. We selected SFGs from the COSMOS2020 catalog and applied a $uv$-domain stacking analysis to their archival Atacama Large Millimeter/submillimeter Array (ALMA) data. Our stacking analysis provides precise measurements of the mean molecular gas mass and size of SFGs. We also applied an image-domain stacking analysis on their \textit{HST} $i$-band and UltraVISTA $J$- and $K_{\rm s}$-band images. Correcting these rest-frame optical sizes using the $R_{\rm half-stellar-light}$-to-$R_{\rm half-stellar-mass}$ conversion at rest 5,000 angstrom, we obtain the stellar mass size of MS galaxies. Across the MS (-0.2 < $\Delta$MS < 0.2), the mean molecular gas fraction of SFGs increases by a factor of $\sim$1.4, while their mean molecular gas depletion time decreases by a factor of $\sim$1.8. The scatter of the MS could thus be caused by variations in both the star formation efficiency and molecular gas fraction of SFGs. The majority of the SFGs lying on the MS have $R_{\rm FIR}$ $\approx$ $R_{\rm stellar}$. Their central regions are subject to large dust attenuation. Starbursts (SBs, $\Delta$MS>0.7) have a mean molecular gas fraction $\sim$2.1 times larger and mean molecular gas depletion time $\sim$3.3 times shorter than MS galaxies. Additionally, they have more compact star-forming regions ($\sim$2.5~kpc for MS galaxies vs. $\sim$1.4~kpc for SBs) and systematically disturbed rest-frame optical morphologies, which is consistent with their association with major-mergers. SBs and MS galaxies follow the same relation between their molecular gas mass and star formation rate surface densities with a slope of $\sim1.1-1.2$, that is, the so-called KS relation. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.11874-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.11874) | **Density distributions, magnetic field structures and fragmentation in  high-mass star formation**  |
|| <mark>H. Beuther</mark>, et al. -- incl., <mark>D. Semenov</mark> |
|*Appeared on*| *2023-11-21*|
|*Comments*| *Accepted for Astronomy & Astrophysics, 14 pages, 14 figures plus appendices, also download option at this https URL*|
|**Abstract**| Methods: Observing the large pc-scale Stokes I mm dust continuum emission with the IRAM 30m telescope and the intermediate-scale (<0.1pc) polarized submm dust emission with the Submillimeter Array toward a sample of 20 high-mass star-forming regions allows us to quantify the dependence of the fragmentation behaviour of these regions depending on the density and magnetic field structures. Results: We infer density distributions n~r^{-p} of the regions with typical power-law slopes p around ~1.5. There is no obvious correlation between the power-law slopes of the density structures on larger clump scales (~1pc) and the number of fragments on smaller core scales (<0.1pc). Comparing the large-scale single-dish density profiles to those derived earlier from interferometric observations at smaller spatial scales, we find that the smaller-scale power-law slopes are steeper, typically around ~2.0. The flattening toward larger scales is consistent with the star-forming regions being embedded in larger cloud structures that do not decrease in density away from a particular core. Regarding the magnetic field, for several regions it appears aligned with filamentary structures leading toward the densest central cores. Furthermore, we find different polarization structures with some regions exhibiting central polarization holes whereas other regions show polarized emission also toward the central peak positions. Nevertheless, the polarized intensities are inversely related to the Stokes I intensities. We estimate magnetic field strengths between ~0.2 and ~4.5mG, and we find no clear correlation between magnetic field strength and the fragmentation level of the regions. Comparison of the turbulent to magnetic energies shows that they are of roughly equal importance in this sample. The mass-to-flux ratios range between ~2 and ~7, consistent with collapsing star-forming regions. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.11534-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.11534) | **Spatial distribution of NH2D in massive star-forming regions**  |
|| Y. Li, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-11-21*|
|*Comments*| *30 pages, 20 figures, 4 tables. Accepted to MNRAS*|
|**Abstract**| To understand the relation between NH$_2$D and its physical environment, we mapped ortho-NH$_2$D $1_{11}^s-1_{01}^a$ at 85.9 GHz toward 24 Galactic late-stage massive star-forming regions with Institut de Radioastronomie Millim$ \'e$trique (IRAM) 30-m telescope. Ortho-NH$_2$D $1_{11}^s-1_{01}^a$ was detected in 18 of 24 sources. Comparing with the distribution of H$^{13}$CN 1-0 as a dense gas tracer and radio recombination line H42$\alpha$, ortho-NH$_2$D $1_{11}^s-1_{01}^a$ present complex and diverse spatial distribution in these targets. 11 of the 18 targets, present a different distribution between ortho-NH$_2$D $1_{11}^s-1_{01}^a$ and H$^{13}$CN 1-0, while no significant difference between these two lines can be found in the other 7 sources, mainly due to limited spatial resolution and sensitivity. Moreover, with H42$\alpha$ tracing massive young stellar objects, ortho-NH$_2$D $1_{11}^s-1_{01}^a$ seems to show a relatively weak emission near the massive young stellar objects. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.11588-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.11588) | **Measuring the Hubble constant with coalescences of binary neutron star  and neutron star-black hole: bright sirens \& dark sirens**  |
|| J. Yu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-11-21*|
|*Comments*| *26 pages, 13 figures, submitted to APJS, comments welcome*|
|**Abstract**| The observations of gravitational wave (GW) provide us a new probe to study the universe. GW events can be used as standard sirens if their redshifts are measured. Normally, stardard sirens can be divided into bright/dark sirens according to whether the redshifts are measured by electromagnetic (EM) counterpart observations. Firstly, we investigate the capability of the 2.5-meter Wide-Field Survey Telescope (WFST) to take follow-up observations of kilonova counterparts. For binary neutron star (BNS) bright sirens, WFST is expected to observe 10-20 kilonovae per year in the second-generation (2G) GW detection era. As for neutron star-black hole (NSBH) mergers, when a BH spin is extremely high and the NS is stiff, the observation rate is $\sim10$ per year. Combining optical and GW observations, the bright sirens are expected to constrain the Hubble constant $H_0$ to $\sim2.8\%$ in five years of observations. As for dark sirens, tidal effects of neutron stars (NSs) during merging time provide us a cosmological model-independent approach to measure the redshifts of GW sources. Then we investigate the applications of tidal effects in redshift measurements. We find in 3G era, the host galaxy groups of around 45\% BNS mergers at $z<0.1$ can be identified through this method, if the EOS is ms1, which is roughly equivalent to the results from luminosity distant constraints. Therefore, tidal effect observations provide a reliable and cosmological model-independent method of identifying BNS mergers' host galaxy groups. Using this method, the BNS/NSBH dark sirens can constrain $H_0$ to 0.2\%/0.3\% over a five-year observation period. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.11909-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.11909) | **Near-Infrared Observations of Outflows and YSOs in the Massive  Star-Forming Region AFGL 5180**  |
|| S. Crowe, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-11-21*|
|*Comments*| *Accepted to Astronomy & Astrophysics (A&A)*|
|**Abstract**| Methods: Broad- and narrow-band imaging of AFGL 5180 was made in the NIR with the LBT, in both seeing-limited ($\sim0.5\arcsec$) and high angular resolution ($\sim0.09\arcsec$) Adaptive Optics (AO) modes, as well as with HST. Archival ALMA continuum data was also utilized. Results: At least 40 jet knots were identified via NIR emission from H$_2$ and [FeII] tracing shocked gas. Bright jet knots outflowing from the central most massive protostar, S4, are detected towards the east of the source and are resolved in fine detail with the AO imaging. Additional knots are distributed throughout the field, likely indicating the presence of multiple driving sources. Sub-millimeter sources detected by ALMA are shown to be grouped in two main complexes, AFGL 5180 M and a small cluster $\sim15\arcsec$ to the south, AFGL 5180 S. From our NIR continuum images we identify YSO candidates down to masses of $\sim 0.1\:M_\odot$. Combined with the sub-mm sources, this yields a surface number density of such YSOs of $N_* \sim 10^3 {\rm pc}^{-2}$ within a projected radius of about 0.1 pc. Such a value is similar to those predicted by models of both Core Accretion from a turbulent clump environment and Competitive Accretion. The radial profile of $N_*$ is relatively flat on scales out to 0.2~pc, with only modest enhancement around the massive protostar inside 0.05~pc. Conclusions: This study demonstrates the utility of high-resolution NIR imaging, in particular with AO, for detecting outflow activity and YSOs in distant regions. The presented images reveal the complex morphology of outflow-shocked gas within the large-scale bipolar flow of a massive protostar, as well as clear evidence for several other outflow driving sources in the region. Finally, this work presents a novel approach to compare the observed YSO surface number density from our study against different models of massive star formation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2311.11832.md
    + _build/html/tmp_2311.11832/./FIR_size.png
    + _build/html/tmp_2311.11832/./ALMA_img.png
    + _build/html/tmp_2311.11832/./Gini_M20.png
exported in  _build/html/2311.11874.md
    + _build/html/tmp_2311.11874/./beta_numcores_st21.png
    + _build/html/tmp_2311.11874/./mass_flux_st21.png
    + _build/html/tmp_2311.11874/./beta_numcores_st21.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# A$^{3}$COSMOS: Dissecting the gas content of star-forming galaxies across the main sequence at 1.2 $\leq z$ < 1.6

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.11832-b31b1b.svg)](https://arxiv.org/abs/2311.11832)<mark>Appeared on: 2023-11-21</mark> -  _20 pages, 17 figures_

</div>
<div id="authors">

T.-M. Wang, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We aim to understand the physical mechanisms that drive star formation in a sample of mass-complete (>10$^{9.5}M_{\odot}$) star-forming galaxies (SFGs) at 1.2 $\leq z$ < 1.6. We selected SFGs from the COSMOS2020 catalog and applied a $uv$-domain stacking analysis to their archival Atacama Large Millimeter/submillimeter Array (ALMA) data. Our stacking analysis provides precise measurements of the mean molecular gas mass and size of SFGs. We also applied an image-domain stacking analysis on their _HST_ $i$-band and UltraVISTA $J$- and $K_{\rm s}$-band images. Correcting these rest-frame optical sizes using the $R_{\rm half-stellar-light}$-to-$R_{\rm half-stellar-mass}$ conversion at rest 5,000 angstrom, we obtain the stellar mass size of MS galaxies. Across the MS (-0.2 < $\Delta$MS < 0.2), the mean molecular gas fraction of SFGs increases by a factor of $\sim$1.4, while their mean molecular gas depletion time decreases by a factor of $\sim$1.8. The scatter of the MS could thus be caused by variations in both the star formation efficiency and molecular gas fraction of SFGs. The majority of the SFGs lying on the MS have $R_{\rm FIR}$ $\approx$ $R_{\rm stellar}$. Their central regions are subject to large dust attenuation. Starbursts (SBs, $\Delta$MS>0.7) have a mean molecular gas fraction $\sim$2.1 times larger and mean molecular gas depletion time $\sim$3.3 times shorter than MS galaxies. Additionally, they have more compact star-forming regions ($\sim$2.5~kpc for MS galaxies vs. $\sim$1.4~kpc for SBs) and systematically disturbed rest-frame optical morphologies, which is consistent with their association with major-mergers. SBs and MS galaxies follow the same relation between their molecular gas mass and star formation rate surface densities with a slope of $\sim1.1-1.2$, that is, the so-called KS relation. 

</div>

<div id="div_fig1">

<img src="tmp_2311.11832/./FIR_size.png" alt="Fig6" width="100%"/>

**Figure 6. -** Circularized effective FIR radius as a function of distance to the MS. Circles and stars show the mean and upper limit FIR sizes for the resolved and unresolved stacked galaxies, respectively. Error bars are the same as in Fig. \ref{fig:gas_frac}, but for the circularized effective FIR radius. Crosses are the mean radio sizes of SFGs at $M_{\star}$ > 10$^{10.5}M_{\odot}$ from \citet[][]{2019A&A...625A.114J}. (*fig:FIR_size*)

</div>
<div id="div_fig2">

<img src="tmp_2311.11832/./ALMA_img.png" alt="Fig11" width="100%"/>

**Figure 11. -** Results in image-domain of our ALMA stacking analysis in $uv$-domain. The maximun values of $\Delta$MS from the lowest to highest stellar mass bins are 1.3, 0.9, 1.0, and 1.1; and the minimum values from the lowest to highest stellar mass bins are -0.7, -0.7, -0.7, and -0.6, respectively. Each panel has a size of 6 arcsec $\times$ 6 arcsec. The synthesized beam of the image is shown in the right-bottom corner of each panel. (*fig:ALMA_img*)

</div>
<div id="div_fig3">

<img src="tmp_2311.11832/./Gini_M20.png" alt="Fig13" width="100%"/>

**Figure 13. -** Morphological classification of our $M_{\star}$> $10^{10}M_{\odot}$ SFGs measured on their _HST_$i$-band images. Circles in the left panel are the Gini and M20 coefficient measured on _HST_$i$-band images of each individual galaxy, while the right panel shows the Gini and M20 coefficient measured for the stacked images of the different stellar mass and $\Delta$MS bin. Orange, pink, green, and dark blue show SBs ($\Delta$MS > 0.7) at 10$^{11}\leq M_{\star}/{\rm M}_{\odot}<10^{12}$, SBs at 10$^{10.5}\leq M_{\star}/{\rm M}_{\odot}<10^{11}$, SBs at 10$^{10}\leq M_{\star}/{\rm M}_{\odot}<10^{10.5}$, and $M_{\star}$> $10^{10}M_{\odot}$ SFGs without SBs ($\Delta$MS < 0.7), respectively. Stars in the left panel are the median Gini and M20 coefficients of SBs in these three stellar mass bins. Gray dashed lines are the morphology classification from \citet{2008ApJ...672..177L}. (*fig:gini_m20*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.11832"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\}{natexlab}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# Density distributions, magnetic field structures and fragmentation in high-mass star formation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.11874-b31b1b.svg)](https://arxiv.org/abs/2311.11874)<mark>Appeared on: 2023-11-21</mark> -  _Accepted for Astronomy & Astrophysics, 14 pages, 14 figures plus appendices, also download option at this https URL_

</div>
<div id="authors">

<mark>H. Beuther</mark>, et al. -- incl., <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** The fragmentation of high-mass star-forming regions depends on a variety of physical parameters, including the density, magnetic field and turbulent gas properties. We evaluate the importance of the density and magnetic field structures in relation to the fragmentation properties during high-mass star formation. Observing the large pc-scale Stokes $I$ mm dust continuum emission with the IRAM 30 m telescope and the intermediate-scale ( $<$ 0.1 pc) polarized submm dust emission with the Submillimeter Array toward a sample of 20 high-mass star-forming regions allows us to quantify the dependence of the fragmentation behaviour of these regions depending on the density and magnetic field structures. Based on the IRAM 30 m data, we infer density distributions $n\propto r^{-p}$ of the regions with typical power-law slopes $p$ around $\sim$ 1.5. There is no obvious correlation between the power-law slopes of the density structures on larger clump scales ( $\sim$ 1 pc) and the number of fragments on smaller core scales ( $<$ 0.1 pc). Comparing the large-scale single-dish density profiles to those derived earlier from interferometric observations at smaller spatial scales, we find that the smaller-scale power-law slopes are steeper, typically around $\sim$ 2.0. The flattening toward larger scales is consistent with the star-forming regions being embedded in larger cloud structures that do not decrease in density away from a particular core. Regarding the magnetic field, for several regions it appears aligned with filamentary structures leading toward the densest central cores. Furthermore, we find different polarization structures with some regions exhibiting central polarization holes whereas other regions show polarized emission also toward the central peak positions. Nevertheless, the polarized intensities are inversely related to the Stokes $I$ intensities, following roughly a power law slope of $\propto S_I^{-0.62}$ . We estimate magnetic field strengths between $\sim$ 0.2 and $\sim$ 4.5 mG, and we find no clear correlation between magnetic field strength and the fragmentation level of the regions.   Comparison of the turbulent to magnetic energies shows that they are of roughly equal importance in this sample. The mass-to-flux ratios range between $\sim$ 2 and $\sim$ 7, consistent with collapsing star-forming regions. Finding no clear correlations between the present-day large-scale density structure, the magnetic field strength and the smaller-scale fragmentation properties of the regions, indicates that the fragmentation of high-mass star-forming regions may not be affected strongly by the initial density profiles and magnetic field properties.   However, considering the limited evolutionary range and spatial scales of the presented CORE analysis, future research directions should include density structure analysis of younger regions that better resemble the initial conditions, as well as connecting the observed intermediate-scale magnetic field structure with the larger-scale magnetic fields of the parental molecular clouds.

</div>

<div id="div_fig1">

<img src="tmp_2311.11874/./beta_numcores_st21.png" alt="Fig1" width="100%"/>

**Figure 1. -** Plot of the ratio of turbulent to magnetic energy $\beta$ versus number of cores. Approximate uncertainties on $\beta$ are around a factor 4. (*beta*)

</div>
<div id="div_fig2">

<img src="tmp_2311.11874/./mass_flux_st21.png" alt="Fig2" width="100%"/>

**Figure 2. -** Plot of the mass-to-flux ratio versus number of cores. For clarity in the logarithmic plotting, the red errorbars are only shown one-sided to higher values. (*mass_flux*)

</div>
<div id="div_fig3">

<img src="tmp_2311.11874/./beta_numcores_st21.png" alt="Fig3" width="100%"/>

**Figure 3. -** Plot of the ratio of turbulent to magnetic energy $\beta$ versus number of cores. Approximate uncertainties on $\beta$ are around a factor 4. (*beta*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.11874"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

324  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
